In [1]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient
import json
import ast
import os
import fitz  
import pytesseract

### start process

In [2]:
endpoint_free = "https://tax1040ocr.cognitiveservices.azure.com/"
key_free = "84f3eba0077d45708e767f8960caf210"
filepath = r"D:\works\silversea-analytics\demo\demo_tax_api\frontend\public\OCR\file2 2024.pdf"

In [8]:
endpoint_pro = "https://henryphamocr.cognitiveservices.azure.com/"
key_pro = "BgokkCOFCdGxs3Nn0u03fOLproraZm4rQzAHXpxunThC8PSnzuR7JQQJ99BDACYeBjFXJ3w3AAALACOG4v7Z"
filepath = r"D:\works\silversea-analytics\demo\demo_tax_api\frontend\public\OCR\file2 2024.pdf"

In [3]:
#   Initial model
client = DocumentIntelligenceClient(endpoint_free, AzureKeyCredential(key_free))

### test OCR new version

In [4]:
def detect_form_type(text: str) -> str:
    text = text.lower()
    if "form 1099-div" in text or "dividends and distributions" in text:
        return "1099-DIV"
    elif "form w-2" in text or "wage and tax statement" in text:
        return "W-2"
    elif "form 1099-int" in text:
        return "1099-INT"
    elif "form 1099-r" in text:
        return "1099-R"
    elif "form 1099-ssa" in text:
        return "SSA-1099"
    elif "schedule k-1" in text:
        return "K-1"
    else:
        return "Unknown"

In [9]:
import fitz  # PyMuPDF

doc = fitz.open(filepath)
toc = doc.get_toc() 

for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        pix = page.get_pixmap(dpi=300)

        img_path = os.path.join(f"page_{page_num + 1}.png")
        pix.save(img_path)

        # OCR ảnh
        text = pytesseract.image_to_string(img_path)

        form_type = detect_form_type(text)
        print(f"📄 Trang {page_num + 1} | Loại biểu mẫu: {form_type}")
        print(f"{'-' * 80}")
        print(f"page: {page_num + 1}")
        print(text)  
        print(f"{'-' * 80}\n")

doc.close()


📄 Trang 1 | Loại biểu mẫu: 1099-DIV
--------------------------------------------------------------------------------
page: 1
MULTI-STRATEGY FUND LLC C/O STATE
STREET BANK AND TRUST CO

BOX 1234

BOSTON, MA 02206

LILY THOMPSON
1234 COASTAL VIEW DR
LOS ANGELES, CA 90045

Payer Code:

3475

If you have questions contact:

Phone:

123-456-7100

CORRECTED (if checked)

PAYER’S name, street address, city or town, state or province, country, ZIP or foreign postal
code, and telephone no.

1a Total ordinary dividends

OMB No. 1545-0110

6 Foreign tax paid

7 Foreign country or U.S. possession

1234

13 State identification
no.

CA

$ 1,542.12
8 Cash liquidationdistributions | 9 Noncash liquidation distributions
$2,565.12 $1,129.12
10 Exempt-interest dividends 11 Specified private activity
bond interest dividends
$3,505.45 $2,512.12
Account number (see instructions) 12 State 14 State tax withheld

$3,165.12

MULTI-STRATEGY FUND LLC C/O STATE $ 5,455.92 9 0 y) A
STREET BANK AND TRUST CO 1b Quali

In [4]:
import re
result_txt = "result_w2.txt"
result_json= []

In [5]:
filepath = r"D:\works\silversea-analytics\demo\demo_tax_api\frontend\public\OCR\file2.pdf"

In [ ]:
print("check11")
with open(filepath, "rb") as f:
    poller = client.begin_analyze_document(
        "prebuilt-tax.us.1099DIV", body=f
    )
tax1040 = poller.result()

check11


In [15]:
print("check22")
print(result_txt)
with open(result_txt, "w", encoding="utf-8") as out_file:
    for i, doc in enumerate(tax1040.documents):
        out_file.write(f"--- Document {i+1} ---\n")
        out_file.write(f"Type: {doc.doc_type}\n")
        out_file.write("All Fields:\n")
        out_file.write(f"{doc.fields}\n\n")

print(f"✅ step1: {result_txt}")

check22
result_w2.txt
✅ step1: result_w2.txt


In [16]:
documents = []
with open(result_txt, "r", encoding="utf-8") as file:
    lines = file.readlines()

current_doc = {}
inside_fields = False
field_lines = []

for line in lines:
    line = line.strip()

    # Bắt đầu document mới
    if line.startswith("--- Document"):
        if field_lines:
            try:
                all_fields_str = "\n".join(field_lines)
                parsed_dict = ast.literal_eval(all_fields_str)
                current_doc["fields"] = parsed_dict
            except Exception as e:
                current_doc["fields"] = {"error": str(e)}
            documents.append(current_doc)
            field_lines = []

        current_doc = {"document_number": int(re.findall(r'\d+', line)[0])}

    elif line.startswith("Type:"):
        current_doc["type"] = line.replace("Type:", "").strip()

    elif line.startswith("All Fields:"):
        inside_fields = True
        field_lines = []

    elif inside_fields:
        if line.startswith("--- Document") or line.startswith("Type:"):
            inside_fields = False
        else:
            field_lines.append(line)

# Thêm document cuối cùng nếu còn
if field_lines:
    try:
        all_fields_str = "\n".join(field_lines)
        current_doc["fields"] = ast.literal_eval(all_fields_str)
    except Exception as e:
        current_doc["fields"] = {"error": str(e)}
    documents.append(current_doc)

print(f"✅ step2: txt to json")

✅ step2: txt to json


In [17]:
documents

[{'document_number': 1,
  'type': 'tax.us.1099DIV.2022',
  'fields': {'TaxYear': {'type': 'string',
    'valueString': '2014',
    'content': '2014',
    'boundingRegions': [{'pageNumber': 1,
      'polygon': [7.3542,
       2.309,
       8.1597,
       2.309,
       8.1597,
       2.6285,
       7.3542,
       2.6285]}],
    'confidence': 0.938,
    'spans': [{'offset': 273, 'length': 4}]},
   'Payer': {'type': 'object',
    'valueObject': {'TIN': {'type': 'string',
      'valueString': '12-3456789',
      'content': '12-3456789',
      'boundingRegions': [{'pageNumber': 1,
        'polygon': [0.8573,
         3.6875,
         1.6181,
         3.6858,
         1.6184,
         3.849,
         0.8576,
         3.8507]}],
      'confidence': 0.915,
      'spans': [{'offset': 514, 'length': 10}]},
     'Name': {'type': 'string',
      'valueString': 'MULTI-STRATEGY FUND LLC C/O STATE',
      'content': 'MULTI-STRATEGY FUND LLC C/O STATE',
      'boundingRegions': [{'pageNumber': 1,
     

In [20]:
# Hàm helper an toàn
def safe_get(d, key, default=""):
    return d.get(key, {}).get("valueString") or d.get(key, {}).get("valueNumber") or d.get(key, {}).get("valueBoolean") or default

def extract_fields(doc):
    fields = doc.get("fields", {})
    document_number = doc.get("document_number", "")

    payer = fields.get("Payer", {}).get("valueObject", {})
    tax_year = fields.get("TaxYear", {}).get("valueString", {})
    recipient = fields.get("Recipient", {}).get("valueObject", {})
    payer_address = payer.get("Address", {}).get("valueAddress", {})
    transactions = fields.get("Transactions", {}).get("valueArray", [])
    first_txn = transactions[0].get("valueObject", {}) if transactions else {}

    state_taxes = first_txn.get("StateTaxesWithheld", {}).get("valueArray", [])
    first_state_tax = state_taxes[0].get("valueObject", {}) if state_taxes else {}

    return {
        "type": "1099-div",
        "tax_year": tax_year,
        "document_number": document_number,
        "Payer_Name": safe_get(payer, "Name"),
        "Payer_TIN": safe_get(payer, "TIN"),
        "Account_Number": safe_get(recipient, "AccountNumber"),

        "Payer_PostalCode": payer_address.get("postalCode", ""),
        "Payer_City": payer_address.get("city", ""),
        "Payer_State": payer_address.get("state", ""),
        
        "Box1a": safe_get(first_txn, "Box1a"),
        "Box1b": safe_get(first_txn, "Box1b"),
        "Box2a": safe_get(first_txn, "Box2a"),
        "Box2b": safe_get(first_txn, "Box2b"),
        "Box2c": safe_get(first_txn, "Box2c"),
        "Box2d": safe_get(first_txn, "Box2d"),

        "Box7": safe_get(first_txn, "Box7"),
        
        "Box9": safe_get(first_txn, "Box9"),
        "Box10": safe_get(first_txn, "Box10"),
        "Box12": safe_get(first_txn, "Box12"),
        "Box13": safe_get(first_txn, "Box13")
    }

# Thực hiện lọc
filtered_result = [extract_fields(doc) for doc in documents]

print(f"✅ Step3: formating JSON")
print(filtered_result)

✅ Step3: formating JSON
[{'type': '1099-div', 'tax_year': '2014', 'document_number': 1, 'Payer_Name': 'MULTI-STRATEGY FUND LLC C/O STATE', 'Payer_TIN': '12-3456789', 'Account_Number': '1234', 'Payer_PostalCode': '02206', 'Payer_City': 'BOSTON', 'Payer_State': 'MA', 'Box1a': 5455.92, 'Box1b': 2234.56, 'Box2a': 1179.56, 'Box2b': 123456, 'Box2c': 1435.56, 'Box2d': 1234.56, 'Box7': 1565.12, 'Box9': 2565.12, 'Box10': 1129.12, 'Box12': 3505.45, 'Box13': 2512.12}]


In [5]:
def OCR_form_w2(filepath):
    print("w2")
    return ""

def OCR_form_1099_div(filepath):

    print("check11")
    with open(filepath, "rb") as f:
        poller = client.begin_analyze_document(
            "prebuilt-tax.us.1099DIV", body=f
        )
    tax1040 = poller.result()

    print("check22")
    print(result_txt)
    with open(result_txt, "w", encoding="utf-8") as out_file:
        for i, doc in enumerate(tax1040.documents):
            out_file.write(f"--- Document {i+1} ---\n")
            out_file.write(f"Type: {doc.doc_type}\n")
            out_file.write("All Fields:\n")
            out_file.write(f"{doc.fields}\n\n")

    print(f"✅ step1: {result_txt}")

    documents = []
    with open(result_txt, "r", encoding="utf-8") as file:
        lines = file.readlines()

    current_doc = {}
    inside_fields = False
    field_lines = []

    for line in lines:
        line = line.strip()

        # Bắt đầu document mới
        if line.startswith("--- Document"):
            if field_lines:
                try:
                    all_fields_str = "\n".join(field_lines)
                    parsed_dict = ast.literal_eval(all_fields_str)
                    current_doc["fields"] = parsed_dict
                except Exception as e:
                    current_doc["fields"] = {"error": str(e)}
                documents.append(current_doc)
                field_lines = []

            current_doc = {"document_number": int(re.findall(r'\d+', line)[0])}

        elif line.startswith("Type:"):
            current_doc["type"] = line.replace("Type:", "").strip()

        elif line.startswith("All Fields:"):
            inside_fields = True
            field_lines = []

        elif inside_fields:
            if line.startswith("--- Document") or line.startswith("Type:"):
                inside_fields = False
            else:
                field_lines.append(line)

    # Thêm document cuối cùng nếu còn
    if field_lines:
        try:
            all_fields_str = "\n".join(field_lines)
            current_doc["fields"] = ast.literal_eval(all_fields_str)
        except Exception as e:
            current_doc["fields"] = {"error": str(e)}
        documents.append(current_doc)

    print(f"✅ step2: txt to json")

    # Hàm helper an toàn
    def safe_get(d, key, default=""):
        return d.get(key, {}).get("valueString") or d.get(key, {}).get("valueNumber") or d.get(key, {}).get("valueBoolean") or default

    def extract_fields(doc):
        fields = doc.get("fields", {})
        document_number = doc.get("document_number", "")

        payer = fields.get("Payer", {}).get("valueObject", {})
        recipient = fields.get("Recipient", {}).get("valueObject", {})
        payer_address = payer.get("Address", {}).get("valueAddress", {})
        transactions = fields.get("Transactions", {}).get("valueArray", [])
        first_txn = transactions[0].get("valueObject", {}) if transactions else {}

        state_taxes = first_txn.get("StateTaxesWithheld", {}).get("valueArray", [])
        first_state_tax = state_taxes[0].get("valueObject", {}) if state_taxes else {}

        return {
            "type": "1099-div",
            "document_number": document_number,
            "Payer_Name": safe_get(payer, "Name"),
            "Payer_TIN": safe_get(payer, "TIN"),
            "Account_Number": safe_get(recipient, "AccountNumber"),

            "Payer_PostalCode": payer_address.get("postalCode", ""),
            "Payer_City": payer_address.get("city", ""),
            "Payer_State": payer_address.get("state", ""),
            
            "Box1a": safe_get(first_txn, "Box1a"),
            "Box1b": safe_get(first_txn, "Box1b"),
            "Box2a": safe_get(first_txn, "Box2a"),
            "Box2b": safe_get(first_txn, "Box2b"),
            "Box2c": safe_get(first_txn, "Box2c"),
            "Box2d": safe_get(first_txn, "Box2d"),

            "Box7": safe_get(first_txn, "Box7"),
            
            "Box9": safe_get(first_txn, "Box9"),
            "Box10": safe_get(first_txn, "Box10"),
            "Box12": safe_get(first_txn, "Box12"),
            "Box13": safe_get(first_txn, "Box13")
        }

    # Thực hiện lọc
    filtered_result = [extract_fields(doc) for doc in documents]

    print(f"✅ Step3: formating JSON")
    return filtered_result
 
 

In [ ]:
OUTPUT_FOLDER = r"D:\works\silversea-analytics\demo\demo_tax_api\frontend\public\OCR"
FORM_HANDLER_MAP = {
    "W-2": OCR_form_w2,
    "1099-DIV": OCR_form_1099_div,
    # "1098": handle_1098,
    # Add more forms here
}
result_json= []

In [7]:
def process_pdf(filepath):
    doc = fitz.open(filepath)
    result_json = []

    for page_num in range(doc.page_count):
        page = doc.load_page(page_num)

        # OCR page
        pix = page.get_pixmap(dpi=300)
        img_path = os.path.join(OUTPUT_FOLDER, f"temp_page_{page_num + 1}.png")
        pix.save(img_path)

        text = pytesseract.image_to_string(img_path)
        form_type = detect_form_type(text)
        print(f"📄 Detected form: {form_type} at page {page_num + 1}")

        if form_type in FORM_HANDLER_MAP:
            print(f"🔧 Processing with handler for {form_type}...")
            single_page_pdf_path = os.path.join(OUTPUT_FOLDER, f"temp_{form_type}_{page_num + 1}.pdf")

            single_doc = fitz.open()
            single_doc.insert_pdf(doc, from_page=page_num, to_page=page_num)
            single_doc.save(single_page_pdf_path)
            single_doc.close()

            output = FORM_HANDLER_MAP[form_type](single_page_pdf_path)

            result_json.append({
                "page_number": page_num + 1,
                "form_type": form_type,
                "data": output
            })

            print(f"✅ Extracted data from {form_type}: {output}")
        else:
            print(f"⚠ No handler defined for form: {form_type}")

    doc.close()
    return result_json

In [ ]:
process_pdf(r"D:\works\silversea-analytics\demo\demo_tax_api\frontend\public\OCR\file2 2024.pdf")

📄 Detected form: 1099-DIV at page 1
🔧 Processing with handler for 1099-DIV...


FzErrorSystem: code=2: cannot remove file 'D:\works\silversea-analytics\demo\demo_tax_api\frontend\public\OCR\temp_1099-DIV_1.pdf': Permission denied

### simple model API

In [11]:
with open(filepath2, "rb") as f:
    poller = client.begin_analyze_document("prebuilt-read", body=f)
    result = poller.result()

- Detect form type

In [9]:
def detect_form_type(text: str) -> str:
    lowered = text.lower()
    if "form w-2" in lowered or "wage and tax statement" in lowered:
        return "W-2"
    if "form 1099-div" in lowered or "dividends and distributions" in lowered:
        return "1099-DIV"
    return "Unknown"

In [5]:
import json
import re
import ast

txt_path = "document_fields_output_w2.txt"  # <- Đặt tên file .txt bạn đã lưu ở đây
json_path = "w2_result.json"

documents = []
with open(txt_path, "r", encoding="utf-8") as file:
    lines = file.readlines()

current_doc = {}
inside_fields = False
field_lines = []

for line in lines:
    line = line.strip()

    # Bắt đầu document mới
    if line.startswith("--- Document"):
        if field_lines:
            try:
                all_fields_str = "\n".join(field_lines)
                parsed_dict = ast.literal_eval(all_fields_str)
                current_doc["fields"] = parsed_dict
            except Exception as e:
                current_doc["fields"] = {"error": str(e)}
            documents.append(current_doc)
            field_lines = []

        current_doc = {"document_number": int(re.findall(r'\d+', line)[0])}

    elif line.startswith("Type:"):
        current_doc["type"] = line.replace("Type:", "").strip()

    elif line.startswith("All Fields:"):
        inside_fields = True
        field_lines = []

    elif inside_fields:
        if line.startswith("--- Document") or line.startswith("Type:"):
            inside_fields = False
        else:
            field_lines.append(line)

# Thêm document cuối cùng nếu còn
if field_lines:
    try:
        all_fields_str = "\n".join(field_lines)
        current_doc["fields"] = ast.literal_eval(all_fields_str)
    except Exception as e:
        current_doc["fields"] = {"error": str(e)}
    documents.append(current_doc)

# Ghi kết quả ra file JSON
with open(json_path, "w", encoding="utf-8") as out:
    json.dump(documents, out, indent=2)

print(f"✅ Chuyển đổi thành công! Đã lưu JSON vào: {json_path}")


✅ Chuyển đổi thành công! Đã lưu JSON vào: w2_result.json


In [8]:
import json

# Đường dẫn tới file JSON đầu vào
input_file = "w2_result.json"
output_file = "w2_filtered.json"

# Load dữ liệu từ file
with open(input_file, "r", encoding="utf-8") as f:
    documents = json.load(f)

# Danh sách các field cần trích xuất
def extract_fields(doc):
    fields = doc.get("fields", {})
    employer = fields.get("Employer", {}).get("valueObject", {})
    employee = fields.get("Employee", {}).get("valueObject", {})

    employee_SocialSecurityNumber = employee.get("SocialSecurityNumber", {}).get("valueString", '')
    print(employee_SocialSecurityNumber)
    employer_address = employer.get("Address", {}).get("valueAddress", {})

    state_infos = fields.get("StateTaxInfos", {}).get("valueArray", [])
    local_infos = fields.get("LocalTaxInfos", {}).get("valueArray", [])

    def safe_get(d, key, default=""):
        return d.get(key, {}).get("valueString") or d.get(key, {}).get("valueNumber") or ""

    return {
        "document_number": doc.get("document_number", ""),
        "SocialSecurityNumber": employee_SocialSecurityNumber,
        "Employer_IdNumber": safe_get(employer, "IdNumber"),
        "Employer_Name": safe_get(employer, "Name"),
        "Employer_Address": (
            f"{employer_address.get('streetAddress', '')} {employer_address.get('city', '')}, "
            f"{employer_address.get('state', '')} {employer_address.get('postalCode', '')}"
            if employer_address else ""
        ),
        "ControlNumber": safe_get(fields, "ControlNumber"),
        "WagesTipsAndOtherCompensation": safe_get(fields, "WagesTipsAndOtherCompensation"),
        "FederalIncomeTaxWithheld": safe_get(fields, "FederalIncomeTaxWithheld"),
        "SocialSecurityWages": safe_get(fields, "SocialSecurityWages"),
        "SocialSecurityTaxWithheld": safe_get(fields, "SocialSecurityTaxWithheld"),
        "MedicareWagesAndTips": safe_get(fields, "MedicareWagesAndTips"),
        "MedicareTaxWithheld": safe_get(fields, "MedicareTaxWithheld"),
        "SocialSecurityTips": safe_get(fields, "SocialSecurityTips"),
        "AllocatedTips": safe_get(fields, "AllocatedTips"),
        "DependentCareBenefits": safe_get(fields, "DependentCareBenefits"),
        "NonQualifiedPlans": safe_get(fields, "NonQualifiedPlans"),
        "IsStatutoryEmployee": safe_get(fields, "IsStatutoryEmployee"),
        "IsRetirementPlan": safe_get(fields, "IsRetirementPlan"),
        "IsThirdPartySickPay": safe_get(fields, "IsThirdPartySickPay"),
        "StateTaxInfos": [
            {
                "State": safe_get(s.get("valueObject", {}), "State"),
                "StateWagesTipsEtc": safe_get(s.get("valueObject", {}), "StateWagesTipsEtc"),
                "StateIncomeTax": safe_get(s.get("valueObject", {}), "StateIncomeTax")
            } for s in state_infos
        ] if state_infos else [],
        "LocalTaxInfos": [
            {
                "LocalWagesTipsEtc": safe_get(l.get("valueObject", {}), "LocalWagesTipsEtc"),
                "LocalIncomeTax": safe_get(l.get("valueObject", {}), "LocalIncomeTax")
            } for l in local_infos
        ] if local_infos else []
    }

# Thực hiện lọc cho tất cả tài liệu
filtered_result = [extract_fields(doc) for doc in documents]

print(filtered_result)

# Ghi ra file JSON mới
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(filtered_result, f, indent=2)

print(f"✅ Đã lọc xong và lưu vào: {output_file}")


123-45-6789
987-65-4321
987-65-4321
987-65-4321
[{'document_number': 1, 'SocialSecurityNumber': '123-45-6789', 'Employer_IdNumber': '12-3456789', 'Employer_Name': 'APPLE INC.', 'Employer_Address': '1234 EMPIRE AVENUE BURBANK, CA 91504', 'ControlNumber': '', 'WagesTipsAndOtherCompensation': 39928.93, 'FederalIncomeTaxWithheld': 9882.68, 'SocialSecurityWages': 39928.93, 'SocialSecurityTaxWithheld': 2475.59, 'MedicareWagesAndTips': 39928.93, 'MedicareTaxWithheld': 578.97, 'SocialSecurityTips': '', 'AllocatedTips': '', 'DependentCareBenefits': '', 'NonQualifiedPlans': '', 'IsStatutoryEmployee': 'false', 'IsRetirementPlan': 'true', 'IsThirdPartySickPay': 'false', 'StateTaxInfos': [{'State': 'NY', 'StateWagesTipsEtc': 39928.93, 'StateIncomeTax': 2728.48}, {'State': '', 'StateWagesTipsEtc': '', 'StateIncomeTax': ''}], 'LocalTaxInfos': [{'LocalWagesTipsEtc': 39928.93, 'LocalIncomeTax': 1605.87}, {'LocalWagesTipsEtc': '', 'LocalIncomeTax': ''}]}, {'document_number': 2, 'SocialSecurityNumber': '

In [ ]:
import json

input_file = "w2_result.json"
output_file = "w2_filtered_split_address.json"

# Load input data
with open(input_file, "r", encoding="utf-8") as f:
    documents = json.load(f)

def extract_fields(doc):
    fields = doc.get("fields", {})
    employer = fields.get("Employer", {}).get("valueObject", {})
    employer_address = employer.get("Address", {}).get("valueAddress", {})
    
    state_infos = fields.get("StateTaxInfos", {}).get("valueArray", [])
    local_infos = fields.get("LocalTaxInfos", {}).get("valueArray", [])

    def safe_get(d, key, default=""):
        return d.get(key, {}).get("valueString") or d.get(key, {}).get("valueNumber") or ""

    result = {
        "document_number": doc.get("document_number", ""),
        "TS": "T",
        "Employer_IdNumber": safe_get(employer, "IdNumber"),
        "Employer_Name": safe_get(employer, "Name"),
        # Tách từng phần địa chỉ
        # "Employer_Address_HouseNumber": employer_address.get("houseNumber", ""),
        # "Employer_Address_Road": employer_address.get("road", ""),
        "Employer_Address_StreetAddress": employer_address.get("streetAddress", ""),
        "Employer_Address_City": employer_address.get("city", ""),
        "Employer_Address_State": employer_address.get("state", ""),
        "Employer_Address_PostalCode": employer_address.get("postalCode", ""),
        # "Employer_Address_Unit": employer_address.get("unit", "")
    }

    # Các trường số/chuỗi khác
    keys = [
        "ControlNumber",
        "WagesTipsAndOtherCompensation",
        "FederalIncomeTaxWithheld",
        "SocialSecurityWages",
        "SocialSecurityTaxWithheld",
        "MedicareWagesAndTips",
        "MedicareTaxWithheld",
        "SocialSecurityTips",
        "AllocatedTips",
        "DependentCareBenefits",
        "NonQualifiedPlans",
        "IsStatutoryEmployee",
        "IsRetirementPlan",
        "IsThirdPartySickPay"
    ]
    for key in keys:
        result[key] = safe_get(fields, key)

    # Lấy StateTaxInfos (nếu có)
    result["StateTaxInfos"] = []
    for s in state_infos:
        val = s.get("valueObject", {})
        result["StateTaxInfos"].append({
            "State": safe_get(val, "State"),
            "StateWagesTipsEtc": safe_get(val, "StateWagesTipsEtc"),
            "StateIncomeTax": safe_get(val, "StateIncomeTax")
        })

    # Lấy LocalTaxInfos (nếu có)
    result["LocalTaxInfos"] = []
    for l in local_infos:
        val = l.get("valueObject", {})
        result["LocalTaxInfos"].append({
            "LocalWagesTipsEtc": safe_get(val, "LocalWagesTipsEtc"),
            "LocalIncomeTax": safe_get(val, "LocalIncomeTax")
        })

    return result

# Áp dụng trích xuất cho tất cả document
filtered_result = [extract_fields(doc) for doc in documents]

# Ghi ra file JSON
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(filtered_result, f, indent=2)

print(f"✅ Đã lưu kết quả tách địa chỉ vào: {output_file}")


✅ Đã lưu kết quả tách địa chỉ vào: w2_filtered_split_address.json


### create excel file

In [17]:
import openpyxl
from openpyxl.utils import get_column_letter

# ======= CẤU HÌNH ========
json_path = "w2_filtered_split_address.json"  # file chứa dữ liệu JSON đã chuẩn hóa
template_excel_path = "w2_format.xlsm"        # file Excel mẫu có sẵn
output_excel_path = "w2_format_filled.xlsx"   # file output để lưu

sheet_name = "W-2 Summary"                    # worksheet cần fill

# ======= LOAD JSON ========
with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# ======= MỞ EXCEL TEMPLATE ========
wb = openpyxl.load_workbook(template_excel_path)
ws = wb[sheet_name]

# ======= CÁC FIELD CẦN FILL ========
main_fields = [
    "Employer_IdNumber", "Employer_Name", "Employer_Address_StreetAddress", "Employer_Address_City",
    "Employer_Address_State", "Employer_Address_PostalCode", "ControlNumber",
    "WagesTipsAndOtherCompensation", "FederalIncomeTaxWithheld", "SocialSecurityWages",
    "SocialSecurityTaxWithheld", "MedicareWagesAndTips", "MedicareTaxWithheld",
    "SocialSecurityTips", "AllocatedTips", "DependentCareBenefits", "NonQualifiedPlans"
]
main_start_row = 9

boolean_fields = ["IsStatutoryEmployee", "IsRetirementPlan", "IsThirdPartySickPay"]
boolean_start_row = 36

state_tax_fields = ["State", "StateWagesTipsEtc", "StateIncomeTax"]
local_tax_fields = ["LocalWagesTipsEtc", "LocalIncomeTax"]
tax_start_row = 40

# ======= GHI DỮ LIỆU ========
for i, doc in enumerate(data):
    col_letter = get_column_letter(i + 2)  # B = 2, C = 3, ...

    # Ghi các field chính
    for j, field in enumerate(main_fields):
        ws[f"{col_letter}{main_start_row + j}"] = doc.get(field, "")

    # Ghi boolean flags
    for j, field in enumerate(boolean_fields):
        ws[f"{col_letter}{boolean_start_row + j}"] = doc.get(field, "")

    # Ghi state tax (chỉ object đầu tiên nếu có)
    state_infos = doc.get("StateTaxInfos")
    state_info = state_infos[0] if isinstance(state_infos, list) and state_infos else {}
    for j, field in enumerate(state_tax_fields):
        ws[f"{col_letter}{tax_start_row + j}"] = state_info.get(field, "")

    # Ghi local tax (chỉ object đầu tiên nếu có)
    local_infos = doc.get("LocalTaxInfos")
    local_info = local_infos[0] if isinstance(local_infos, list) and local_infos else {}
    for j, field in enumerate(local_tax_fields):
        ws[f"{col_letter}{tax_start_row + len(state_tax_fields) + j}"] = local_info.get(field, "")

# ======= LƯU FILE MỚI ========
wb.save(output_excel_path)
print(f"✅ Đã fill dữ liệu vào file: {output_excel_path}")


✅ Đã fill dữ liệu vào file: w2_format_filled.xlsx


### 1099-DIV

In [ ]:
endpoint_pro = "https://henryphamocr.cognitiveservices.azure.com/"
key_pro = "BgokkCOFCdGxs3Nn0u03fOLproraZm4rQzAHXpxunThC8PSnzuR7JQQJ99BDACYeBjFXJ3w3AAALACOG4v7Z"
filepath = r"C:\Users\Admin\Downloads\20181231 ADP Year End W2.pdf"

In [1]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient


endpoint_pro = "https://henryphamocr.cognitiveservices.azure.com/"
key_pro = "BgokkCOFCdGxs3Nn0u03fOLproraZm4rQzAHXpxunThC8PSnzuR7JQQJ99BDACYeBjFXJ3w3AAALACOG4v7Z"
filepath = r"C:\Users\Admin\Downloads\1099-div.png"

document_intelligence_client  = DocumentIntelligenceClient(
    endpoint=endpoint_pro, credential=AzureKeyCredential(key_pro)
)


In [4]:



with open(filepath, "rb") as f:
    poller = document_intelligence_client.begin_analyze_document(
        "prebuilt-tax.us.1099DIV", body=f
    )
tax1040 = poller.result()




In [5]:
with open("document_fields_output_1099_DIV.txt", "w", encoding="utf-8") as out_file:
    for i, doc in enumerate(tax1040.documents):
        out_file.write(f"--- Document {i+1} ---\n")
        out_file.write(f"Type: {doc.doc_type}\n")
        out_file.write("All Fields:\n")
        out_file.write(f"{doc.fields}\n\n")

print("✅ Đã ghi kết quả vào file: document_fields_output_1099_DIV.txt")

✅ Đã ghi kết quả vào file: document_fields_output_1099_DIV.txt


In [6]:
import json
import re
import ast

txt_path = "document_fields_output_1099_DIV.txt"  # <- Đặt tên file .txt bạn đã lưu ở đây
json_path = "1099_DIV_result.json"

documents = []
with open(txt_path, "r", encoding="utf-8") as file:
    lines = file.readlines()

current_doc = {}
inside_fields = False
field_lines = []

for line in lines:
    line = line.strip()

    # Bắt đầu document mới
    if line.startswith("--- Document"):
        if field_lines:
            try:
                all_fields_str = "\n".join(field_lines)
                parsed_dict = ast.literal_eval(all_fields_str)
                current_doc["fields"] = parsed_dict
            except Exception as e:
                current_doc["fields"] = {"error": str(e)}
            documents.append(current_doc)
            field_lines = []

        current_doc = {"document_number": int(re.findall(r'\d+', line)[0])}

    elif line.startswith("Type:"):
        current_doc["type"] = line.replace("Type:", "").strip()

    elif line.startswith("All Fields:"):
        inside_fields = True
        field_lines = []

    elif inside_fields:
        if line.startswith("--- Document") or line.startswith("Type:"):
            inside_fields = False
        else:
            field_lines.append(line)

# Thêm document cuối cùng nếu còn
if field_lines:
    try:
        all_fields_str = "\n".join(field_lines)
        current_doc["fields"] = ast.literal_eval(all_fields_str)
    except Exception as e:
        current_doc["fields"] = {"error": str(e)}
    documents.append(current_doc)

# Ghi kết quả ra file JSON
with open(json_path, "w", encoding="utf-8") as out:
    json.dump(documents, out, indent=2)

print(f"✅ Chuyển đổi thành công! Đã lưu JSON vào: {json_path}")


✅ Chuyển đổi thành công! Đã lưu JSON vào: 1099_DIV_result.json


### Đang tìm list mapping

In [8]:
import json

# Input/output
input_file = "1099_DIV_result.json"
output_file = "1099_DIV_result2.json"

# Load dữ liệu
with open(input_file, "r", encoding="utf-8") as f:
    documents = json.load(f)

# Hàm helper an toàn
def safe_get(d, key, default=""):
    return d.get(key, {}).get("valueString") or d.get(key, {}).get("valueNumber") or d.get(key, {}).get("valueBoolean") or default

def extract_fields(doc):
    fields = doc.get("fields", {})
    document_number = doc.get("document_number", "")

    payer = fields.get("Payer", {}).get("valueObject", {})
    recipient = fields.get("Recipient", {}).get("valueObject", {})
    payer_address = payer.get("Address", {}).get("valueAddress", {})
    transactions = fields.get("Transactions", {}).get("valueArray", [])
    first_txn = transactions[0].get("valueObject", {}) if transactions else {}

    state_taxes = first_txn.get("StateTaxesWithheld", {}).get("valueArray", [])
    first_state_tax = state_taxes[0].get("valueObject", {}) if state_taxes else {}

    return {
        "document_number": document_number,
        "Payer_Name": safe_get(payer, "Name"),
        "Payer_TIN": safe_get(payer, "TIN"),
        "Account_Number": safe_get(recipient, "AccountNumber"),

        "Payer_PostalCode": payer_address.get("postalCode", ""),
        "Payer_City": payer_address.get("city", ""),
        "Payer_State": payer_address.get("state", ""),
        
        "Box1a": safe_get(first_txn, "Box1a"),
        "Box1b": safe_get(first_txn, "Box1b"),
        "Box2a": safe_get(first_txn, "Box2a"),
        "Box2b": safe_get(first_txn, "Box2b"),
        "Box2c": safe_get(first_txn, "Box2c"),
        "Box2d": safe_get(first_txn, "Box2d"),

        "Box7": safe_get(first_txn, "Box7"),
        
        "Box9": safe_get(first_txn, "Box9"),
        "Box10": safe_get(first_txn, "Box10"),
        "Box12": safe_get(first_txn, "Box12"),
        "Box13": safe_get(first_txn, "Box13")
    }

# Thực hiện lọc
filtered_result = [extract_fields(doc) for doc in documents]

# Lưu kết quả
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(filtered_result, f, indent=2)

print(f"✅ Đã lọc xong và lưu vào: {output_file}")


✅ Đã lọc xong và lưu vào: 1099_DIV_result2.json


In [10]:
import json
import openpyxl

# ======= CẤU HÌNH ========
json_path = "1099_DIV_result2.json"  # file JSON đầu vào
template_excel_path = "1099_div_format.xlsx"  # file Excel mẫu
output_excel_path = "1099_div_filled.xlsx"  # file Excel sau khi fill

sheet_name = "Brokerage Summary (Sch B, D)"

# ======= LOAD DỮ LIỆU ========
with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# ======= MỞ EXCEL TEMPLATE ========
wb = openpyxl.load_workbook(template_excel_path)
ws = wb[sheet_name]

# ======= GHI DỮ LIỆU CHO MỖI DOCUMENT ========
for i, doc in enumerate(data):
    col_letter = openpyxl.utils.get_column_letter(i + 4)  # D = 4, E = 5,...

    # Ghi thông tin chung
    ws[f"{col_letter}8"] = "2-DIV"
    ws[f"{col_letter}9"] = doc.get("Payer_Name", "")
    ws[f"{col_letter}10"] = doc.get("Payer_TIN", "")
    ws[f"{col_letter}11"] = doc.get("Account_Number", "")

    # Box1a → Box2d (ô 35-40)
    ws[f"{col_letter}35"] = doc.get("Box1a", "")
    ws[f"{col_letter}36"] = doc.get("Box1b", "")
    ws[f"{col_letter}37"] = doc.get("Box2a", "")
    ws[f"{col_letter}38"] = doc.get("Box2b", "")
    ws[f"{col_letter}39"] = doc.get("Box2c", "")
    ws[f"{col_letter}40"] = doc.get("Box2d", "")

    # Box7
    ws[f"{col_letter}42"] = doc.get("Box7", "")

    # Box9 → Box13 (ô 45-48)
    ws[f"{col_letter}45"] = doc.get("Box9", "")
    ws[f"{col_letter}46"] = doc.get("Box10", "")
    ws[f"{col_letter}47"] = doc.get("Box12", "")
    ws[f"{col_letter}48"] = doc.get("Box13", "")

# ======= LƯU FILE ========
wb.save(output_excel_path)
print(f"✅ Đã fill dữ liệu vào file: {output_excel_path}")


✅ Đã fill dữ liệu vào file: 1099_div_filled.xlsx


d:\works\silversea-analytics\demo\demo_tax_api\venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
